In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-09-07 01:35:51 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-09-07 01:35:51 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-09-07 01:35:51 __main__ INFO     transformers.__version__='4.55.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-09-07 01:35:54 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-07 01:35:54 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-07 01:35:54 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-09-07 01:35:54 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-09-07 01:35:55 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-09-07 01:35:55 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-09-07 01:35:55 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-09-07 01:35:55 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-09-07 01:35:55 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-09-07 01:36:45 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-09-07 01:36:45 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-09-07 01:36:45 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask, SelectOrderTask

#################################################################################
# TASK_CLS = SelectOrderTask
# prompt_template_idx = 1
TASK_CLS = SelectOneTask
prompt_template_idx = 3
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR, 
        "selection", 
        # "profession.json"
        # "nationality.json"
        "objects.json"
    )
)

print(select_task)

SelectOneTask: (different objects)
Categories: fruit(15), vehicle(15), furniture(15), animal(15), music instrument(15), clothing(15), electronics(15), sport equipment(15), kitchen appliance(15), vegetable(14), building(15), office supply(15), bathroom item(15), flower(15), tree(15), jewelry(15)



In [6]:
sample = select_task.get_random_sample(
    mt = mt,
    option_style=OPTION_STYLE,
    prompt_template_idx=prompt_template_idx,
    obj_idx=2,
    # category="actor",
    # category="Brazil"
    category="fruit",
    filter_by_lm_prediction=False,
)

print(sample)
print(sample.prompt())

from src.selection.utils import verify_correct_option
# sample.prompt_template = select_prof.prompt_templates[3]
print(f'"{sample.prompt()}"', ">>", sample.obj)

verify_correct_option(
    mt=mt,
    target=sample.obj,
    options=sample.options,
    input=sample.prompt()
)

Raspberry -> Mango (2): ['Tablet', 'Dishwasher', 'Mango', 'Ambulance', 'Bench', 'Cedar']
Options: Tablet, Dishwasher, Mango, Ambulance, Bench, Cedar.
Which among these objects mentioned above is a fruit?
Answer:
"Options: Tablet, Dishwasher, Mango, Ambulance, Bench, Cedar.
Which among these objects mentioned above is a fruit?
Answer:" >> Mango


(True,
 [PredictedToken(token=' Mango', prob=0.90234375, logit=22.25, token_id=91963, metadata=None),
  PredictedToken(token=' The', prob=0.044921875, logit=19.25, token_id=578, metadata=None),
  PredictedToken(token=' Among', prob=0.021240234375, logit=18.5, token_id=22395, metadata=None),
  PredictedToken(token=' A', prob=0.006103515625, logit=17.25, token_id=362, metadata=None),
  PredictedToken(token=' Option', prob=0.003265380859375, logit=16.625, token_id=7104, metadata=None)],
 OrderedDict([(91963,
               (1,
                PredictedToken(token=' Mango', prob=0.90234375, logit=22.25, token_id=91963, metadata=None))),
              (20423,
               (21,
                PredictedToken(token=' Amb', prob=0.00026702880859375, logit=14.125, token_id=20423, metadata=None))),
              (58403,
               (92,
                PredictedToken(token=' Tablet', prob=1.329183578491211e-05, logit=11.125, token_id=58403, metadata=None))),
              (49268,
          

In [9]:
from src.tokens import prepare_input, TokenizerOutput, find_token_range
from src.selection.functional import verify_head_patterns

sample_tokenized = prepare_input(
    prompts=sample.prompt(),
    tokenizer=mt.tokenizer,
)
def locate_with_delim(prompt, option):
    st = prompt.index(option)
    return prompt[st : st + len(option) + 1]

attn_pattern = verify_head_patterns(  # noqa
    prompt=sample.prompt(),
    tokenized_prompt=sample_tokenized,
    options=(
        [
            locate_with_delim(sample.prompt(), opt)
            for opt in sample.options
        ]
    ),
    # options=sample.options,
    pivot=sample.subj,
    mt=mt,
    heads=[(35, 19)],
    # heads=[(35, 19)],
    # generate_full_answer=True,
    query_index=-1,
    ablate_possible_ans_info_from_options=True,
)

2025-09-07 01:37:02 src.selection.functional DEBUG    Tablet, | opt_range=(3, 5) | bare_opt_range=(1, 3)
2025-09-07 01:37:02 src.selection.functional DEBUG    opt=" Tablet," | bare_opt=" Tablet,"
2025-09-07 01:37:02 src.selection.functional DEBUG    After adjusted bare_opt_range=(3, 5): bare_opt=" Tablet,"
2025-09-07 01:37:02 src.selection.functional DEBUG    Dishwasher, | opt_range=(5, 8) | bare_opt_range=(1, 4)
2025-09-07 01:37:02 src.selection.functional DEBUG    opt=" Dishwasher," | bare_opt=" Dishwasher,"
2025-09-07 01:37:02 src.selection.functional DEBUG    After adjusted bare_opt_range=(5, 8): bare_opt=" Dishwasher,"
2025-09-07 01:37:02 src.selection.functional DEBUG    Mango, | opt_range=(8, 10) | bare_opt_range=(1, 3)
2025-09-07 01:37:02 src.selection.functional DEBUG    opt=" Mango," | bare_opt=" Mango,"
2025-09-07 01:37:02 src.selection.functional DEBUG    After adjusted bare_opt_range=(8, 10): bare_opt=" Mango,"


2025-09-07 01:37:02 src.selection.functional DEBUG    Ambulance, | opt_range=(10, 13) | bare_opt_range=(1, 4)
2025-09-07 01:37:02 src.selection.functional DEBUG    opt=" Ambulance," | bare_opt=" Ambulance,"
2025-09-07 01:37:02 src.selection.functional DEBUG    After adjusted bare_opt_range=(10, 13): bare_opt=" Ambulance,"
2025-09-07 01:37:03 src.selection.functional DEBUG    Bench, | opt_range=(13, 15) | bare_opt_range=(1, 3)
2025-09-07 01:37:03 src.selection.functional DEBUG    opt=" Bench," | bare_opt=" Bench,"
2025-09-07 01:37:03 src.selection.functional DEBUG    After adjusted bare_opt_range=(13, 15): bare_opt=" Bench,"
2025-09-07 01:37:03 src.selection.functional DEBUG    Cedar. | opt_range=(15, 17) | bare_opt_range=(1, 3)
2025-09-07 01:37:03 src.selection.functional DEBUG    opt=" Cedar.
" | bare_opt=" Cedar."
2025-09-07 01:37:03 src.selection.functional DEBUG    After adjusted bare_opt_range=(15, 17): bare_opt=" Cedar."
2025-09-07 01:37:03 src.selection.functional DEBUG    Predi

In [10]:
from src.selection.functional import get_patches_to_verify_independent_enrichment
from src.functional import patch_with_baukit, interpret_logits
from src.selection.utils import get_first_token_id

patches = get_patches_to_verify_independent_enrichment(
    prompt = sample.prompt(),
    options=(
        [
            locate_with_delim(sample.prompt(), opt)
            for opt in sample.options
        ]
    ),
    pivot=sample.subj,
    mt=mt,
    tokenized_prompt=sample_tokenized,
)

patched_out = patch_with_baukit(
    mt=mt,
    inputs=sample_tokenized,
    patches=patches,
)
logits = patched_out.logits[:, -1, :]
interpret_logits(
    logits=logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[get_first_token_id(opt, mt.tokenizer) for opt in sample.options],
)

2025-09-07 01:37:25 src.selection.functional DEBUG    Tablet, | opt_range=(3, 5) | bare_opt_range=(1, 3)
2025-09-07 01:37:25 src.selection.functional DEBUG    opt=" Tablet," | bare_opt=" Tablet,"
2025-09-07 01:37:25 src.selection.functional DEBUG    After adjusted bare_opt_range=(3, 5): bare_opt=" Tablet,"


2025-09-07 01:37:25 src.selection.functional DEBUG    Dishwasher, | opt_range=(5, 8) | bare_opt_range=(1, 4)
2025-09-07 01:37:25 src.selection.functional DEBUG    opt=" Dishwasher," | bare_opt=" Dishwasher,"
2025-09-07 01:37:25 src.selection.functional DEBUG    After adjusted bare_opt_range=(5, 8): bare_opt=" Dishwasher,"
2025-09-07 01:37:26 src.selection.functional DEBUG    Mango, | opt_range=(8, 10) | bare_opt_range=(1, 3)
2025-09-07 01:37:26 src.selection.functional DEBUG    opt=" Mango," | bare_opt=" Mango,"
2025-09-07 01:37:26 src.selection.functional DEBUG    After adjusted bare_opt_range=(8, 10): bare_opt=" Mango,"
2025-09-07 01:37:26 src.selection.functional DEBUG    Ambulance, | opt_range=(10, 13) | bare_opt_range=(1, 4)
2025-09-07 01:37:26 src.selection.functional DEBUG    opt=" Ambulance," | bare_opt=" Ambulance,"
2025-09-07 01:37:26 src.selection.functional DEBUG    After adjusted bare_opt_range=(10, 13): bare_opt=" Ambulance,"
2025-09-07 01:37:26 src.selection.functional D

([PredictedToken(token=' Mango', prob=0.36328125, logit=15.6875, token_id=91963, metadata=None),
  PredictedToken(token=' The', prob=0.08642578125, logit=14.25, token_id=578, metadata=None),
  PredictedToken(token=' A', prob=0.06298828125, logit=13.9375, token_id=362, metadata=None),
  PredictedToken(token=' None', prob=0.0263671875, logit=13.0625, token_id=2290, metadata=None),
  PredictedToken(token=' Apple', prob=0.0247802734375, logit=13.0, token_id=8325, metadata=None)],
 OrderedDict([(91963,
               (1,
                PredictedToken(token=' Mango', prob=0.36328125, logit=15.6875, token_id=91963, metadata=None))),
              (20423,
               (183,
                PredictedToken(token=' Amb', prob=0.00021457672119140625, logit=8.25, token_id=20423, metadata=None))),
              (57748,
               (343,
                PredictedToken(token=' Cedar', prob=6.961822509765625e-05, logit=7.125, token_id=57748, metadata=None))),
              (49268,
               

In [11]:
LIMIT = 256

test_set = []
while len(test_set) < LIMIT:
    logger.info(f"Collecting test samples: {len(test_set) + 1}/{LIMIT}")
    sample = select_task.get_random_sample(
        mt = mt,
        option_style=OPTION_STYLE,
        prompt_template_idx=prompt_template_idx,
        n_distractors=N_DISTRACTORS,
        filter_by_lm_prediction=True,
    )
    test_set.append(sample)

2025-09-07 01:37:37 __main__ INFO     Collecting test samples: 1/256


2025-09-07 01:37:37 __main__ INFO     Collecting test samples: 2/256
2025-09-07 01:37:38 __main__ INFO     Collecting test samples: 3/256
2025-09-07 01:37:38 __main__ INFO     Collecting test samples: 4/256
2025-09-07 01:37:38 __main__ INFO     Collecting test samples: 5/256
2025-09-07 01:37:38 __main__ INFO     Collecting test samples: 6/256
2025-09-07 01:37:38 __main__ INFO     Collecting test samples: 7/256
2025-09-07 01:37:39 __main__ INFO     Collecting test samples: 8/256
2025-09-07 01:37:39 __main__ INFO     Collecting test samples: 9/256
2025-09-07 01:37:39 __main__ INFO     Collecting test samples: 10/256
2025-09-07 01:37:39 __main__ INFO     Collecting test samples: 11/256
2025-09-07 01:37:39 __main__ INFO     Collecting test samples: 12/256
2025-09-07 01:37:40 __main__ INFO     Collecting test samples: 13/256
2025-09-07 01:37:40 __main__ INFO     Collecting test samples: 14/256
2025-09-07 01:37:40 __main__ INFO     Collecting test samples: 15/256
2025-09-07 01:37:40 __main__

In [12]:
ind_enrich_results = []
for i, sample in enumerate(test_set):
    logger.info(f"Processing test sample: {i+1}/{len(test_set)}")
    print(sample)
    print(sample.prompt())
    sample_tokenized = prepare_input(
        prompts=sample.prompt(),
        tokenizer=mt.tokenizer,
    )
    interested_tokens = [
        get_first_token_id(opt, mt.tokenizer) for opt in sample.options
    ]
    # clean run
    clean_output = patch_with_baukit(
        mt=mt,
        inputs=sample_tokenized,
        patches=[],
    )
    clean_logits = clean_output.logits[:, -1, :]
    clean_pred, clean_track = interpret_logits(
        logits=clean_logits,
        tokenizer=mt.tokenizer,
        interested_tokens=interested_tokens,
    )
    logger.info(f"clean pred={[str(pred) for pred in clean_pred]}")
    logger.info(f"clean track={clean_track}")
    logger.info("-"*75)

    # get the patches
    patches = get_patches_to_verify_independent_enrichment(
        prompt = sample.prompt(),
        options=(
            [
                locate_with_delim(sample.prompt(), opt)
                for opt in sample.options
            ]
        ),
        # options=sample.options,
        pivot=sample.subj,
        mt=mt,
        tokenized_prompt=sample_tokenized,
    )

    # patched run
    patched_out = patch_with_baukit(
        mt=mt,
        inputs=sample_tokenized,
        patches=patches,
    )
    int_logits = patched_out.logits[:, -1, :]
    int_pred, int_track = interpret_logits(
        logits=int_logits,
        tokenizer=mt.tokenizer,
        interested_tokens=[get_first_token_id(opt, mt.tokenizer) for opt in sample.options],
    )
    logger.info(f"int pred={[str(pred) for pred in int_pred]}")
    logger.info(f"int track={int_track}")

    ind_enrich_results.append({
        "sample": sample,
        "interested_tokens": interested_tokens,
        "clean": {
            "pred": clean_pred,
            "track": clean_track,
        },
        "int": {
            "pred": int_pred,
            "track": int_track,
        },
    })
    logger.info("=" * 100)
    

2025-09-07 01:38:26 __main__ INFO     Processing test sample: 1/256


Scarf -> Jeans (3): ['Toaster', 'Warehouse', 'Harp', 'Jeans', 'Lotion', 'Boxing gloves']
Options: Toaster, Warehouse, Harp, Jeans, Lotion, Boxing gloves.
Which among these objects mentioned above is a clothing?
Answer:
2025-09-07 01:38:26 __main__ INFO     clean pred=['" Jeans"[82507] (p=0.801, logit=22.000)', '" The"[578] (p=0.084, logit=19.750)', '" Among"[22395] (p=0.058, logit=19.375)', '" JE"[71430] (p=0.011, logit=17.750)', '" Har"[5340] (p=0.008, logit=17.375)']
2025-09-07 01:38:26 __main__ INFO     clean track=OrderedDict([(82507, (1, PredictedToken(token=' Jeans', prob=0.80078125, logit=22.0, token_id=82507, metadata=None))), (5340, (5, PredictedToken(token=' Har', prob=0.00787353515625, logit=17.375, token_id=5340, metadata=None))), (2057, (45, PredictedToken(token=' To', prob=0.0001354217529296875, logit=13.3125, token_id=2057, metadata=None))), (72683, (48, PredictedToken(token=' Boxing', prob=0.00012683868408203125, logit=13.25, token_id=72683, metadata=None))), (445, (88,

In [13]:
passed_cases = []
faile_cases = []

for result in ind_enrich_results:
    sample = result["sample"]
    int_track = result["int"]["track"]
    if int_track[list(int_track.keys())[0]][1].token_id == sample.ans_token_id:
        passed_cases.append(result)
    else:
        faile_cases.append(result)

accuracy = len(passed_cases) / len(ind_enrich_results)
logger.info(f"Independent enrichment accuracy: {accuracy*100:.2f}%")

2025-09-07 01:47:46 __main__ INFO     Independent enrichment accuracy: 80.08%


In [21]:
from src.tokens import prepare_input, TokenizerOutput, find_token_range
from src.selection.functional import verify_head_patterns

sample = passed_cases[15]["sample"]
sample_tokenized = prepare_input(
    prompts=sample.prompt(),
    tokenizer=mt.tokenizer,
)
def locate_with_delim(prompt, option):
    st = prompt.index(option)
    return prompt[st : st + len(option) + 1]

attn_pattern = verify_head_patterns(  # noqa
    prompt=sample.prompt(),
    tokenized_prompt=sample_tokenized,
    # options=(
    #     [
    #         locate_with_delim(sample.prompt(), opt)
    #         for opt in sample.options
    #     ]
    # ),
    options=sample.options,
    pivot=sample.subj,
    mt=mt,
    heads=[(35, 19)],
    # heads=[(35, 19)],
    # generate_full_answer=True,
    query_index=-1,
    ablate_possible_ans_info_from_options=True,
)

2025-09-07 01:50:59 src.selection.functional DEBUG    Tape | opt_range=(3, 4) | bare_opt_range=(1, 2)
2025-09-07 01:50:59 src.selection.functional DEBUG    opt=" Tape" | bare_opt=" Tape"
2025-09-07 01:50:59 src.selection.functional DEBUG    After adjusted bare_opt_range=(3, 4): bare_opt=" Tape"


2025-09-07 01:51:00 src.selection.functional DEBUG    Iris | opt_range=(5, 6) | bare_opt_range=(1, 2)
2025-09-07 01:51:00 src.selection.functional DEBUG    opt=" Iris" | bare_opt=" Iris"
2025-09-07 01:51:00 src.selection.functional DEBUG    After adjusted bare_opt_range=(5, 6): bare_opt=" Iris"
2025-09-07 01:51:00 src.selection.functional DEBUG    Helmet | opt_range=(7, 8) | bare_opt_range=(1, 2)
2025-09-07 01:51:00 src.selection.functional DEBUG    opt=" Helmet" | bare_opt=" Helmet"
2025-09-07 01:51:00 src.selection.functional DEBUG    After adjusted bare_opt_range=(7, 8): bare_opt=" Helmet"
2025-09-07 01:51:00 src.selection.functional DEBUG    Shorts | opt_range=(9, 10) | bare_opt_range=(1, 2)
2025-09-07 01:51:00 src.selection.functional DEBUG    opt=" Shorts" | bare_opt=" Shorts"
2025-09-07 01:51:00 src.selection.functional DEBUG    After adjusted bare_opt_range=(9, 10): bare_opt=" Shorts"
2025-09-07 01:51:00 src.selection.functional DEBUG    Bookshelf | opt_range=(11, 13) | bare_op

In [22]:
passed_cases[15]["int"]["pred"]

[PredictedToken(token=' None', prob=0.330078125, logit=15.3125, token_id=2290, metadata=None),
 PredictedToken(token=' A', prob=0.12158203125, logit=14.3125, token_id=362, metadata=None),
 PredictedToken(token=' The', prob=0.06103515625, logit=13.625, token_id=578, metadata=None),
 PredictedToken(token=' Basketball', prob=0.034912109375, logit=13.0625, token_id=47589, metadata=None),
 PredictedToken(token=' Skate', prob=0.03076171875, logit=12.9375, token_id=97796, metadata=None)]